# Overview
- Decode Whisper fine-tuned and Whisper-Flamingo on MuAViC Es
- Default beam size is 1, but the paper uses beam size of 5

# Setup

In [1]:
# Verify the correct versions are loaded
import numpy; print(numpy.__version__)
import tensorboard; print(tensorboard.__version__)

1.22.0
2.14.0


In [21]:
!nvidia-smi

Wed Feb 12 11:04:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               On  | 00000000:17:00.0 Off |                  Off |
| 30%   25C    P8              20W / 230W |   6001MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import sys
sys.path.insert(0, 'av_hubert/fairseq')
import os
import json
import argparse
import numpy as np
import torch
from torch import nn
from scipy.io import wavfile
import pandas as pd
import whisper
from tqdm import tqdm
import sacrebleu
from fairseq.scoring.wer import WerScorer, WerScorerConfig
#from spec_augment import spec_augment
from utils import (
    load_data,
    WhisperVideoCollatorWithPadding,
)
from utils_batch_samplers import LengthBatchSampler
from whisper_ft_muavic_video import MuavicVideoDataset
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer, seed_everything

/usr/users/roudi/vtenvs/anaconda3/envs/muavic/lib/python3.8/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-02-12 10:55:14 | INFO | lightning_fabric.utilities.seed | Seed set to 3407


In [4]:
# verify that we are using the local whisper
print(whisper.__file__)

/data/sls/u/meng/roudi/whisper-flamingo/whisper/__init__.py


# Decode Whisper Fine-tuned Small for Es ASR

In [10]:
# general configs
lang = 'es'
model_type = 'small'

noise_snr = 0 # noisy
# noise_snr = 1000 # clean

noise_fn = 'noise/babble/lrs3/test.tsv'
checkpoint_root = ''

beam_size = 1
# beam_size = 15

fp16 = True if torch.cuda.is_available() else False

In [11]:
# asr config
checkpoint = 'models/whisper_multi-all_small.pt' # audio
modalities = 'asr'
use_av_hubert_encoder = 0
av_fusion = 'early'
video_model_path=None,
av_hubert_path=None,


In [12]:
def setup_data():
    checkpoint_path = os.path.join(checkpoint_root, checkpoint)
    SAMPLE_RATE = 16000
    SEED = 3407
    seed_everything(SEED, workers=True)

    audio_transcript_pair_list = load_data(480000, 350, [lang], muavic_root='',
                                           include_audio_lens=True, lrs2=False, vc2=True)
    test_dataset =  audio_transcript_pair_list['test']
    test_dataset = [[i[0], i[1].replace('/data/sls/scratch/roudi/datasets/muavic/',
                                        ''),
                                         i[2], i[3]] for i in test_dataset]# fix paths
    
    #test_dataset = [[i[0], i[1],i[2], i[3]] for i in test_dataset] # fix paths
    multilingual = True if 'large' in model_type or 'en' not in model_type else False
    print("Multilingual: {}", multilingual)
    tokenizer = whisper.tokenizer.get_tokenizer(multilingual=True, task='transcribe') # NOTE: use transcribe token as hack
    print("Multilingual tokenizer : {}".format(multilingual))
    special_token_set = set(tokenizer.special_tokens.values())
    dataset = MuavicVideoDataset(test_dataset,
                                    tokenizer,
                                    SAMPLE_RATE,
                                    model_type,
                                    max_length=None if checkpoint_path else SAMPLE_RATE * 30,
                                    spec_augment="", # no spec augment
                                    noise_prob=1 if noise_snr == 0 else 0, # TODO: implement non-0 snr
                                    noise_fn = noise_fn,
                                    train=False, # video center crop, no flip,
                                    noise_snr=  noise_snr,
                                )

    length_sorter = LengthBatchSampler(batch_bins=SAMPLE_RATE * 40 if checkpoint_path and \
                                    beam_size == 1 else 1,
                                shapes=[i[3] for i in test_dataset],
                                sort_in_batch='descending',
                                sort_batch='descending',
                                drop_last=False)

    dataloader = torch.utils.data.DataLoader(dataset,
                        num_workers=8,
                        collate_fn=WhisperVideoCollatorWithPadding(),
                        batch_sampler=length_sorter
                        )
    return tokenizer, special_token_set, dataloader
tokenizer, special_token_set, dataloader = setup_data()

2025-02-12 10:55:35 | INFO | lightning_fabric.utilities.seed | Seed set to 3407


Removed 136 samples with missing video (before filtering lengths)
Removed 0 samples with missing text
Total hours train : 204.29574397569445
es train 124717 123236
Removed 0 samples with missing video (before filtering lengths)
Removed 0 samples with missing text
Total hours valid : 1.450880486111111
es valid 905 898
Removed 0 samples with missing video (before filtering lengths)
Removed 0 samples with missing text
Total hours test : 1.728941527777778
es test 1012 1012
Total data lengths
123236
898
1012
Multilingual: {} True
Multilingual tokenizer : True
Dataloader max length : None
Loaded 1 noise wavs


In [13]:
def load_model():
    checkpoint_path = os.path.join(checkpoint_root, checkpoint)
    print("load model" + checkpoint_path)
    print("Loading Whisper")
    whisper_model = whisper.load_model(model_type,
                                    video=True if av_fusion == 'separate' else 0,
                                    video_model_path=video_model_path,
                                    av_hubert_path=av_hubert_path,
                                    av_hubert_encoder=use_av_hubert_encoder,
                                    av_fusion=av_fusion,
                                    add_gated_x_attn=1 if av_fusion == 'separate' else 0)

    if checkpoint_path is not None:
        print("Loading checkpoint")
        state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        print(state_dict.keys())
        state_dict = state_dict['state_dict']
        state_dict_updated = {k[6:]: v  for k, v in state_dict.items()} # remove 'model.'
        try: # newer models have learnable scaler init 1
            whisper_model.load_state_dict(state_dict_updated)
        except BaseException as e:
            print(str(e))
            print("Loading weights with strict=False")
            whisper_model.load_state_dict(state_dict_updated, strict=False)

    if fp16 and use_av_hubert_encoder == 1:
        whisper_model.encoder.video_projection_scalar.half()
        whisper_model.encoder.video_model.half()
        model_to_num_layers = {'small': 12, 'medium': 24, 'large-v2': 32}
        if av_fusion == 'separate':
            for i in range(model_to_num_layers[model_type]):
                whisper_model.decoder.blocks[i].attn_gate.data = whisper_model.decoder.blocks[i].attn_gate.half()
                whisper_model.decoder.blocks[i].ff_gate.data = whisper_model.decoder.blocks[i].ff_gate.half()
    return whisper_model
whisper_model = load_model()

load modelmodels/whisper_multi-all_small.pt
Loading Whisper


Whisper dropout rate : 0.0
Loading checkpoint
dict_keys(['state_dict'])


In [14]:
def decode_model(num_batches = -1):
    if beam_size == 1: # greedy search
        options = whisper.DecodingOptions(language=lang, fp16=fp16, without_timestamps=True)
    else:
        options = whisper.DecodingOptions(language=lang, fp16=fp16, without_timestamps=True, beam_size=beam_size)
    hypo, refs = [], []
    whisper_model.eval() # AV-HuBERT batch norm and dropout
    for i, b in enumerate(tqdm(dataloader)):
        if fp16:
            input_ids = b["input_ids"].half().cuda()
            video = b["video"].half().cuda()
        else:
            if torch.cuda.is_available():
              input_ids = b["input_ids"].cuda()
              video = b["video"].cuda()
            else:
              input_ids = b["input_ids"]
              video = b["video"]
        labels = b["labels"]
        with torch.no_grad():
            if modalities == "avsr":
                results = whisper_model.decode(input_ids, options, video)
            elif modalities == "asr":
                results = whisper_model.decode(input_ids, options, video, test_a=True)
            else:
                raise NotImplementedError

            for r, l in zip(results, labels):
                hypo.append(r.text)
                print('HYPO: {}'.format(r.text))

                l[l == -100] = tokenizer.eot
                ref = tokenizer.decode([t for t in l if t.item() not in special_token_set])
                refs.append(ref)
                print('REF: {}'.format(ref))
        if i + 1 == num_batches:
            break
    return hypo, refs

In [15]:
hypo, refs = decode_model(num_batches=-1)

  1%|          | 1/140 [00:02<06:28,  2.80s/it]

HYPO: música
REF:  música aplausos y ahora os voy a enseñar cómo el agua puede hacer más sonidos que el que hace cuando sale del grifo


  1%|▏         | 2/140 [00:04<04:33,  1.98s/it]

HYPO: pónganse en situación estas chicas han experimentado llevan mucha carga muchos trastornos postraumático debido a un éxodo a una tierra que nadie les había prometido a mí esto como asesinaban a sus seres queridos a mí esto como incendiaba no bombardeaban sus casas
REF:  pónganse en situación estas chicas llevan mucha carga muchos trastornos postraumáticos debido a un éxodo a una tierra que nadie les había prometido han visto cómo asesinaban a sus seres queridos han visto cómo incendiaban o bombardeaban sus casas otras desgraciadamente también han sido testigos o víctimas de violaciones en su particular trasiego desde afganistán irak y siria camino a europa o camino en este caso de alemania


  2%|▏         | 3/140 [00:05<04:04,  1.79s/it]

...

 99%|█████████▉| 139/140 [01:48<00:00,  3.21it/s]

HYPO: adivináis
REF:  a tu bar
HYPO: no es posible no preguntar
REF:  para eso sirven las matemáticas
HYPO: y lo fueron y lo han hecho
REF:  y nos fue muy bien
HYPO: te parás con los que te prendan
REF:  si te pagan estupendo
HYPO: los vecinos y guías
REF:  los vecinos lo miraban
HYPO: y le dije tranquilo
REF:  tiene mucho sentido
HYPO: pero no lo encontré
REF:  pero no lo demostró
HYPO: nos partimos
REF:  y por qué el cine
HYPO: por qué viajar
REF:  por qué viajaste sola
HYPO: hasta aquí
REF:  ay las películas
HYPO: casi igual
REF:  casi iguales
HYPO: es un proyecto que tiene aficionados
REF:  escribes genial
HYPO: para qué sirve la voz
REF:  para qué sirve el amor
HYPO: yo no lo veo
REF:  qué es la vida
HYPO: no no
REF:  no no
HYPO: ya pasaron ya
REF:  qué ha pasado este año
HYPO: me encanta hacerlo
REF:  me encanta observar
HYPO: cómo me la pido a verlo
REF:  para qué sirve la vida misma
HYPO: qué es decir de la poesía
REF:  para qué sirve la poesía
HYPO: no había ni un dron
REF:  n

100%|██████████| 140/140 [01:48<00:00,  3.48it/s]

HYPO: y no es gracia
REF:  y nina me miró así
HYPO: se te ha desbastecado
REF:  risas aunque sea de plástico
HYPO: gracias
REF:  gracias
HYPO: y la hacía muy bien
REF:  la vida
HYPO: me encanta
REF:  me encanta
HYPO: y dijo kiké
REF:  dijo quique
HYPO: vos
REF:  vos
HYPO: y eso qué es
REF:  y eso qué es
HYPO: increíble
REF:  increíble
HYPO: a quién le va la pan
REF:  por qué me van a pagar
HYPO: fantástico miedo
REF:  contad conmigo
HYPO: cómo estás hoy
REF:  cómo estás hoy
HYPO: pero no
REF:  pero moncho
HYPO: pero no es así
REF:  pero no es así
HYPO: criando un robot
REF:  podría usar triángulos
HYPO: no hay que perderlo
REF:  por más que nos duela
HYPO: no sé
REF:  no os fieis
HYPO: la diferencia es de verdad
REF:  preguntó pepe
HYPO: ya se fue
REF:  y así fue
HYPO: muchas gracias
REF:  muchas gracias
HYPO: por qué
REF:  con qué
HYPO: y me dijo me miraba
REF:  y mi hija me miraba
HYPO: ya no estaría todo
REF:  te apuntarías
HYPO: hasta ahora
REF:  hasta pronto
HYPO: qué es lo de la 

100%|██████████| 140/140 [01:48<00:00,  1.29it/s]


In [16]:
def get_score():
    scorer = WerScorer(
        WerScorerConfig(
            wer_tokenizer="13a",
            wer_remove_punct=True,
            wer_char_level=False,
            wer_lowercase=True
        )
    )
    for h, r in zip(hypo, refs):
        scorer.add_string(ref=r, pred=h)
        wer = scorer.score()
    print("WER : %.4f" % wer)
get_score()

WER : 46.6788


##### Noisy: the paper reports WER of 41.7 for Es, but that is with beam size = 5. WER of 46.67 is with beam size 1.
##### Clean: the paper reports WER of 9.5 for Es, but that is with beam size = 5. WER of 10.16 is with beam size 1.

# Decode mWhisper-Flamingo Small for Es ASR

In [17]:
# avsr config
checkpoint= 'models/whisper-flamingo_multi-all_small.pt'
modalities = 'avsr'
use_av_hubert_encoder = 1
av_fusion = 'separate'
video_model_path = 'models/mavhubert_only_weights.pt'
av_hubert_path =  'av_hubert/avhubert'

In [18]:
lang = 'es'
tokenizer, special_token_set, dataloader = setup_data()
whisper_model = load_model()

2025-02-12 10:57:28 | INFO | lightning_fabric.utilities.seed | Seed set to 3407


Removed 136 samples with missing video (before filtering lengths)
Removed 0 samples with missing text
Total hours train : 204.29574397569445
es train 124717 123236
Removed 0 samples with missing video (before filtering lengths)
Removed 0 samples with missing text
Total hours valid : 1.450880486111111
es valid 905 898
Removed 0 samples with missing video (before filtering lengths)
Removed 0 samples with missing text
Total hours test : 1.728941527777778
es test 1012 1012
Total data lengths
123236
898
1012
Multilingual: {} True
Multilingual tokenizer : True
Dataloader max length : None
Loaded 1 noise wavs
load modelmodels/whisper-flamingo_multi-all_small.pt
Loading Whisper
Whisper dropout rate : 0.0
Loading AV-HuBERT encoder
Using AV-HuBERT encoder with parameters: 325142504
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
A

In [19]:
# decode on noisy audio
hypo, refs = decode_model(num_batches=-1)

  1%|          | 1/140 [00:03<07:08,  3.08s/it]

HYPO: música de guitarra en el fondo de la guitarra hay una canción que se llama la música de la palma
REF:  música aplausos y ahora os voy a enseñar cómo el agua puede hacer más sonidos que el que hace cuando sale del grifo


  1%|▏         | 2/140 [00:05<06:09,  2.68s/it]

HYPO: pónganse en situación estas chicas han experimentado llevan mucha carga muchos trastornos postraumático debido a un éxodo a una tierra que nadie les había prometido a mí esto como asesinaban a sus seres queridos a mí esto como incendiaba no bombardeaban sus casas otras desgraciadamente
REF:  pónganse en situación estas chicas llevan mucha carga muchos trastornos postraumáticos debido a un éxodo a una tierra que nadie les había prometido han visto cómo asesinaban a sus seres queridos han visto cómo incendiaban o bombardeaban sus casas otras desgraciadamente también han sido testigos o víctimas de violaciones en su particular trasiego desde afganistán irak y siria camino a europa o camino en este caso de alemania


  2%|▏         | 3/140 [00:07<05:44,  2.52s/it]

...

HYPO: le dije no
REF:  le dije 'no
HYPO: y qué es el día
REF:  y qué es el día
HYPO: yo no me reojo
REF:  que en lo que veo
HYPO: pero a la confusión sentí
REF:  pero algo feo sentí
HYPO: por qué caminas en su lugar
REF:  por qué caminaste sola
HYPO: el asuelo es lo primero que pasa
REF:  era todo lo que había hecho
HYPO: que cualquier cosa se aprende
REF:  más que cualquier diamante
HYPO: por qué quieren que lo quieran
REF:  para qué quieren peluquera
HYPO: una de las mejores ideas que he tenido
REF:  cuál es la mejor pieza que puedo usar
HYPO: qué sirve hacer esto
REF:  para qué sirve hacer esto
HYPO: y tu proyecto de lado
REF:  y tu proyecto de helado
HYPO: qué pasó acá
REF:  qué pasó acá
HYPO: no se ha cambiado
REF:  no tengan miedo
HYPO: y darme un fin de cliente
REF:  y carmen no tiene clientes
HYPO: no se me ve como un árbol
REF:  y yo le digo claro
HYPO: me gusta lo que veo
REF:  o está en lo que veo
HYPO: necesito algo fuerte
REF:  necesito algo fuerte
HYPO: y si no hubiera la

 99%|█████████▉| 139/140 [02:38<00:00,  2.41it/s]

HYPO: adurránse
REF:  a tu bar
HYPO: no sirve de las malas manchas
REF:  para eso sirven las matemáticas
HYPO: y lo fueron viendo
REF:  y nos fue muy bien
HYPO: este para por si estupendas
REF:  si te pagan estupendo
HYPO: los vecinos lo viven
REF:  los vecinos lo miraban
HYPO: y le dije tranquilo
REF:  tiene mucho sentido
HYPO: pero no lo vemos todos
REF:  pero no lo demostró
HYPO: por qué el terapeuta
REF:  y por qué el cine
HYPO: por qué viajar solo
REF:  por qué viajaste sola
HYPO: ah y las películas
REF:  ay las películas
HYPO: casi igual
REF:  casi iguales
HYPO: esas que llevas a kenia
REF:  escribes genial
HYPO: para qué sirve la robótica
REF:  para qué sirve el amor
HYPO: en la vida
REF:  qué es la vida
HYPO: no no
REF:  no no
HYPO: y ha pasado en diario
REF:  qué ha pasado este año
HYPO: me encanta observar
REF:  me encanta observar
HYPO: cómo la vida es más
REF:  para qué sirve la vida misma
HYPO: qué es decir ver la poesía
REF:  para qué sirve la poesía
HYPO: no había nombre

100%|██████████| 140/140 [02:38<00:00,  1.13s/it]

HYPO: y niña me miró así
REF:  y nina me miró así
HYPO: es el trate básico
REF:  risas aunque sea de plástico
HYPO: gracias
REF:  gracias
HYPO: la vida
REF:  la vida
HYPO: me encanta
REF:  me encanta
HYPO: y dijo quíquen
REF:  dijo quique
HYPO: vos
REF:  vos
HYPO: y eso qué es
REF:  y eso qué es
HYPO: increíble
REF:  increíble
HYPO: lo que me va a dar para acá
REF:  por qué me van a pagar
HYPO: fantástico miedo
REF:  contad conmigo
HYPO: cómo estás hoy
REF:  cómo estás hoy
HYPO: y los niños
REF:  pero moncho
HYPO: pero no es así
REF:  pero no es así
HYPO: criarnos
REF:  podría usar triángulos
HYPO: no es marketing o cuidad
REF:  por más que nos duela
HYPO: no es feliz
REF:  no os fieis
HYPO: pero no yo pierdo
REF:  preguntó pepe
HYPO: ya se fue
REF:  y así fue
HYPO: muchas gracias
REF:  muchas gracias
HYPO: por qué
REF:  con qué
HYPO: y me dijo me miraba
REF:  y mi hija me miraba
HYPO: ya está bien
REF:  te apuntarías
HYPO: hasta arriba
REF:  hasta pronto
HYPO: qué es lo de la tarea
RE

In [20]:
get_score()

WER : 38.6676


##### Noisy: the paper reports WER of 33.6 for Es, but that is with beam size = 5. WER of 38.7 is with beam size 1.